<a href="https://colab.research.google.com/github/unknownguy24/Deep-Learning/blob/main/CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
print(tf.__version__)

2.9.2


In [4]:
fashion_mnist=keras.datasets.fashion_mnist             #importing the fashion_mnist dataset from keras datasets

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()               #downloads images and labels we split into train and test

4422102/4422102 [==============================] - 0s 0us/step


In [6]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()   #train test images labels

In [7]:
def build_model(hp):
    
    model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),  #creating range of integers with the name conv_1_filter where your min value is 32 and max value is 128 and step is 16
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),                 #kernel size is also called as filter size 
        activation='relu',                                                      #the value 32 and 128 tells us the number of filters in that layer
        input_shape=(28,28,1)                                                   #keras tuner will help us find out what number of filters will be good in each layer
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),   #keras tuner will go through each number of filter size values
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),                 #hp.choice chooses between 3 and 5
        activation='relu'
    ),
    keras.layers.Flatten(),                                                     #Flatten the layer
    keras.layers.Dense(                                                         #Creating the dense layer
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),    #Number of nodes/neurons in the dense layer
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')                                #Adding last dense layer with 10 o/p nodes 
    ])
  
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
    return model

In [10]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 23s]
val_accuracy: 0.7730000019073486

Best val_accuracy So Far: 0.887666642665863
Total elapsed time: 00h 03m 10s


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 32)        51232     
                                                                 
 flatten (Flatten)           (None, 15488)             0         
                                                                 
 dense (Dense)               (None, 48)                743472    
                                                                 
 dense_1 (Dense)             (None, 10)                490       
                                                                 
Total params: 795,834
Trainable params: 795,834
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.2277 - accuracy: 0.9147 - val_loss: 0.3582 - val_accuracy: 0.8792
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1973 - accuracy: 0.9260 - val_loss: 0.3420 - val_accuracy: 0.8843
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1659 - accuracy: 0.9381 - val_loss: 0.3498 - val_accuracy: 0.8890
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1421 - accuracy: 0.9476 - val_loss: 0.4093 - val_accuracy: 0.8890
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1200 - accuracy: 0.9564 - val_loss: 0.4146 - val_accuracy: 0.8925
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1062 - accuracy: 0.9613 - val_loss: 0.5132 - val_accuracy: 0.8905
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0898 - accuracy: 0.9675 - val_loss: 0.5282 - val_accuracy